In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.database import database
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = database.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [6]:
######################################################
########## TIME IN PROXIMITY BEHAVIOR ################
######################################################

#
cohort.track_type = 'features'
cohort.exclude_huddles = True
cohort.recompute = True

#
behavior_name = 'pairwise_proximity'

# select animals
cohort.animal_ids = [0,1,2,3,4,5]

all_pairs = [(a, b) for idx, a in enumerate(cohort.animal_ids) for b in cohort.animal_ids[idx + 1:]]
print ("all_pairs: ", all_pairs)

#
for pair in all_pairs:

    cohort.animal_ids = [pair[0],pair[1]]
    
    #
    cohort.get_pairwise_interaction_time(pair[0],
                                         pair[1])

    #
    cohort.format_behavior()

    #
    ethogram.generate_ethogram_hourly( 
                                      behavior_name,
                                      cohort,
                                      cohort.exclude_huddles)

all_pairs:  [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


100%|██████████| 1500/1500 [00:03<00:00, 484.37it/s]


In [9]:
##############################################
########## SHOW PAIRWISE PROXIMITY ###########
##############################################

#
for pair in all_pairs:

    cohort.animal_ids = [pair[0],pair[1]]
    
    cohort.exclude_huddles = True
    ethogram.plot_ethogram_hourly(
                                 behavior_name,
                                 cohort,
                                 cohort.exclude_huddles,
                                 vmax=50)
    